<a href="https://colab.research.google.com/github/kamikazekartik/cs744_assignments/blob/master/project/VGG11_Cifar10_ksreenivasan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Baseline performance and timings for training LeNet on MNIST/EMNIST.

Before running the notebook, go to Runtime -> Change Runtime Type in the menu and set Hardware Accelerator to GPU. **Make sure you change it back** when finished to avoid being penalized by Colab.

You can use the dataset variable to decide if to run on MNIST or EMNIST (EMNIST will be slightly slower since the training data is significantly larger)

Each cell runs with different precision settings

Strange behavior right now:
1. Full precision is the fastest.
2. Half precision is about half a second slower per epoch.
3. AMP is still slower by another whole second.

Code is based on [here](https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html).

In [1]:
import time
import os, random
import copy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import torch
from torch.utils.data import random_split
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision import models
from typing import Union, List, Dict, Any, cast
import logging

__all__ = [
    'VGG', 'vgg11', 'vgg11_bn', 'vgg13', 'vgg13_bn', 'vgg16', 'vgg16_bn',
    'vgg19_bn', 'vgg19',
]


model_urls = {
    'vgg11': 'https://download.pytorch.org/models/vgg11-bbd30ac9.pth',
    'vgg13': 'https://download.pytorch.org/models/vgg13-c768596a.pth',
    'vgg16': 'https://download.pytorch.org/models/vgg16-397923af.pth',
    'vgg19': 'https://download.pytorch.org/models/vgg19-dcbb9e9d.pth',
    'vgg11_bn': 'https://download.pytorch.org/models/vgg11_bn-6002323d.pth',
    'vgg13_bn': 'https://download.pytorch.org/models/vgg13_bn-abd245e5.pth',
    'vgg16_bn': 'https://download.pytorch.org/models/vgg16_bn-6c64b313.pth',
    'vgg19_bn': 'https://download.pytorch.org/models/vgg19_bn-c79401a0.pth',
}


class VGG(nn.Module):

    def __init__(
        self,
        features: nn.Module,
        num_classes: int = 1000,
        init_weights: bool = True
    ) -> None:
        super(VGG, self).__init__()
        self.features = features
        self.avgpool = nn.AdaptiveAvgPool2d((7, 7))
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, num_classes),
        )
        if init_weights:
            self._initialize_weights()

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

    def _initialize_weights(self) -> None:
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)


def make_layers(cfg: List[Union[str, int]], batch_norm: bool = False) -> nn.Sequential:
    layers: List[nn.Module] = []
    in_channels = 3
    for v in cfg:
        if v == 'M':
            layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
        else:
            v = cast(int, v)
            conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=1)
            if batch_norm:
                layers += [conv2d, nn.BatchNorm2d(v), nn.ReLU(inplace=True)]
            else:
                layers += [conv2d, nn.ReLU(inplace=True)]
            in_channels = v
    return nn.Sequential(*layers)


cfgs: Dict[str, List[Union[str, int]]] = {
    'A': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'B': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'D': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'E': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
}


def _vgg(arch: str, cfg: str, batch_norm: bool, pretrained: bool, progress: bool, **kwargs: Any) -> VGG:
    if pretrained:
        kwargs['init_weights'] = False
    model = VGG(make_layers(cfgs[cfg], batch_norm=batch_norm), **kwargs)
    if pretrained:
        state_dict = load_state_dict_from_url(model_urls[arch],
                                              progress=progress)
        model.load_state_dict(state_dict)
    return model


def vgg11(pretrained: bool = False, progress: bool = True, **kwargs: Any) -> VGG:
    r"""VGG 11-layer model (configuration "A") from
    `"Very Deep Convolutional Networks For Large-Scale Image Recognition" <https://arxiv.org/pdf/1409.1556.pdf>`._

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _vgg('vgg11', 'A', False, pretrained, progress, **kwargs)


def vgg11_bn(pretrained: bool = False, progress: bool = True, **kwargs: Any) -> VGG:
    r"""VGG 11-layer model (configuration "A") with batch normalization
    `"Very Deep Convolutional Networks For Large-Scale Image Recognition" <https://arxiv.org/pdf/1409.1556.pdf>`._

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _vgg('vgg11_bn', 'A', True, pretrained, progress, **kwargs)


def vgg13(pretrained: bool = False, progress: bool = True, **kwargs: Any) -> VGG:
    r"""VGG 13-layer model (configuration "B")
    `"Very Deep Convolutional Networks For Large-Scale Image Recognition" <https://arxiv.org/pdf/1409.1556.pdf>`._

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _vgg('vgg13', 'B', False, pretrained, progress, **kwargs)


def vgg13_bn(pretrained: bool = False, progress: bool = True, **kwargs: Any) -> VGG:
    r"""VGG 13-layer model (configuration "B") with batch normalization
    `"Very Deep Convolutional Networks For Large-Scale Image Recognition" <https://arxiv.org/pdf/1409.1556.pdf>`._

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _vgg('vgg13_bn', 'B', True, pretrained, progress, **kwargs)


def vgg16(pretrained: bool = False, progress: bool = True, **kwargs: Any) -> VGG:
    r"""VGG 16-layer model (configuration "D")
    `"Very Deep Convolutional Networks For Large-Scale Image Recognition" <https://arxiv.org/pdf/1409.1556.pdf>`._

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _vgg('vgg16', 'D', False, pretrained, progress, **kwargs)


def vgg16_bn(pretrained: bool = False, progress: bool = True, **kwargs: Any) -> VGG:
    r"""VGG 16-layer model (configuration "D") with batch normalization
    `"Very Deep Convolutional Networks For Large-Scale Image Recognition" <https://arxiv.org/pdf/1409.1556.pdf>`._

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _vgg('vgg16_bn', 'D', True, pretrained, progress, **kwargs)


def vgg19(pretrained: bool = False, progress: bool = True, **kwargs: Any) -> VGG:
    r"""VGG 19-layer model (configuration "E")
    `"Very Deep Convolutional Networks For Large-Scale Image Recognition" <https://arxiv.org/pdf/1409.1556.pdf>`._

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _vgg('vgg19', 'E', False, pretrained, progress, **kwargs)


def vgg19_bn(pretrained: bool = False, progress: bool = True, **kwargs: Any) -> VGG:
    r"""VGG 19-layer model (configuration 'E') with batch normalization
    `"Very Deep Convolutional Networks For Large-Scale Image Recognition" <https://arxiv.org/pdf/1409.1556.pdf>`._

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _vgg('vgg19_bn', 'E', True, pretrained, progress, **kwargs)

In [7]:
logging.basicConfig()
logger = logging.getLogger()
logger.setLevel(logging.INFO)

def seed_experiment(seed=0):
    # seed = 1234
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    # TODO: Do we need deterministic in cudnn ? Double check
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    print("Seeded everything with seed: {}".format(seed))

# seed experiment
seed_experiment(42)

# Options:
use_amp = False
use_half_all = True
use_half_conv = False
use_half_lin = False
dataset = 'Cifar10'
PRELOAD = False # decides if we should use pytorch's dataloader or just preload into a python list

# Make sure we're using a GPU, and report what GPU it is.
# (Otherwise this would run **forever**)
if torch.cuda.is_available():
  print("using "+torch.cuda.get_device_name(0))
else:
  print('No GPU available (enable it?), quitting.')
  exit()
device = torch.device("cuda:0")

# Set up dataset:
batch_size = 64
test_batch_size = 1000

class ToHalfTensor(object):
    """Convert Tensors to HalfTensors"""
    def __init__(self, use_half):
      self.use_half = use_half

    def __call__(self, img):
        """
        Args:
            Tensor, use_half

        Returns:
            Half precision typecast tensor if use_half=True
              else: do nothing
        """
        if self.use_half:
          img = img.half()

        return img


def get_dataloader(dataset="MNIST", use_half=False, PRELOAD=False):
  if dataset == 'MNIST':
      train_set = torchvision.datasets.MNIST('./data', train=True, download=True,
                        transform=transforms.Compose([
                            transforms.ToTensor(),
                            transforms.Normalize((0.1307,), (0.3081,)),
                            ToHalfTensor(use_half),
                        ]))
      test_set = torchvision.datasets.MNIST('./data', train=False, transform=transforms.Compose([
                            transforms.ToTensor(),
                            transforms.Normalize((0.1307,), (0.3081,)),
                            ToHalfTensor(use_half),
                        ]))
  elif dataset == 'EMNIST':
    train_set = torchvision.datasets.EMNIST('./data', split="digits", train=True, download=True,
                        transform=transforms.Compose([
                            transforms.ToTensor(),
                            transforms.Normalize((0.1307,), (0.3081,)),
                            ToHalfTensor(use_half),
                        ]))
    test_set = torchvision.datasets.EMNIST('./data', split="digits", train=False, transform=transforms.Compose([
                            transforms.ToTensor(),
                            transforms.Normalize((0.1307,), (0.3081,)),
                            ToHalfTensor(use_half),
                        ]))
    
  elif dataset == 'Cifar10':
    transform_train = transforms.Compose([
            transforms.RandomCrop(32, padding=4),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
            ToHalfTensor(use_half),
        ])

    transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
        ToHalfTensor(use_half),
    ])

    train_set = torchvision.datasets.CIFAR10(
        root='./data', train=True, download=True, transform=transform_train)

    test_set = torchvision.datasets.CIFAR10(
        root='./data', train=False, download=True, transform=transform_test)

  train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size,
                                             shuffle=True, num_workers=2)
  test_loader = torch.utils.data.DataLoader(test_set, batch_size=test_batch_size,
                                            shuffle=False, num_workers=2)
  
  preloaded_train_loader = []
  if PRELOAD:
    # nothing fancy. Just preload into a list
    for batch_idx, (inputs, labels) in enumerate(train_loader):
      preloaded_train_loader.append((inputs, labels))
    train_loader = preloaded_train_loader

  return (train_loader, test_loader)


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 4 * 4, 120)  # 6*6 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

# test function:
def test(dataset, model, device, test_loader, criterion):
  class_correct = list(0. for i in range(10))
  class_total = list(0. for i in range(10))
  if dataset in ['EMNIST', 'MNIST']:
      classes = [str(i) for i in range(10)]
  elif dataset == 'Cifar10':
    classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog',
               'horse', 'ship', 'truck')

  model.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad():
      for data, target in test_loader:
          data, target = data.to(device), target.to(device)
          output = model(data)
          _, predicted = torch.max(output, 1)
          c = (predicted == target).squeeze()

          test_loss = criterion(output, target).item()
          pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
          correct += pred.eq(target.view_as(pred)).sum().item()

          for image_index in range(test_batch_size):
              label = target[image_index]
              class_correct[label] += c[image_index].item()
              class_total[label] += 1

  test_loss /= len(test_loader.dataset)

  for i in range(10):
      logger.info('Accuracy of %5s : %2d %%' % (
          classes[i], 100 * class_correct[i] / class_total[i]))

  logger.info('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
      test_loss, correct, len(test_loader.dataset),
      100. * correct / len(test_loader.dataset)))
  
  return 100.0 * correct/len(test_loader.dataset)

# train method
def train(model, optimizer, criterion, scaler, train_loader, use_amp, epoch=0):

  for batch_idx, (inputs, labels) in enumerate(train_loader): # Iterating through the train loader
    inputs, labels = inputs.to(device), labels.to(device)
    optimizer.zero_grad()            # Reset the gradient in every iteration
    with torch.cuda.amp.autocast(enabled=use_amp):
      outputs = model(inputs)
      loss = criterion(outputs,labels) # Loss forward pass
    scaler.scale(loss).backward()      # Loss backward pass
    scaler.step(optimizer)
    scaler.update()                    # Update all the parameters by the given learning rule
    # optimizer.zero_grad()              # set_to_none=True here can modestly improve performance
  
    if batch_idx % 500 == 0:
      logger.info('Train Epoch: {} [{} ({:.0f}%)]\tLoss: {:.6f}'.format(
          epoch, batch_idx * len(inputs),
          100. * batch_idx / len(train_loader), loss.item()))


  return loss.item()

def run_experiment(MAX_EPOCHS=3):
  epoch_list = [0]
  loss_epoch_list = [-1]
  epoch_train_time_list = [-1]
  total_train_time_list = [-1]
  lr_list = [-1]
  test_acc_list = []

  # get data
  train_loader, test_loader = get_dataloader(dataset, use_half=use_half_all, PRELOAD=PRELOAD)
  
  if dataset in ['MNIST', 'EMNIST']:
    model = Net().to(device)
  elif dataset == 'Cifar10':
    model = vgg11().to(device)
  else:
    logger.info("BAD DATASET!!!")
    exit()
  
  if use_half_all:
    model.half()
  criterion = nn.CrossEntropyLoss()
  curr_lr = 0.01
  optimizer = optim.SGD(model.parameters(), lr=curr_lr, momentum=0.9)
  scaler = torch.cuda.amp.GradScaler(enabled=use_amp)
  total_training_time = 0

  # check accuracy before training
  test_acc = test(dataset, model, device, test_loader, criterion)
  test_acc_list.append(test_acc)

  for epoch in range(1, MAX_EPOCHS):
    start_time = time.time()
    last_epoch_loss = train(model, optimizer, criterion, scaler, train_loader, use_amp, epoch)
    end_time = time.time()
    epoch_training_time = end_time - start_time
    total_training_time += epoch_training_time
    epoch_list.append(epoch)
    epoch_train_time_list.append(epoch_training_time)
    total_train_time_list.append(total_training_time)
    lr_list.append(curr_lr)
    test_acc = test(dataset, model, device, test_loader, criterion)
    test_acc_list.append(test_acc)
    loss_epoch_list.append(last_epoch_loss)

    # cut learning rate in half every 20 epochs
    if epoch % 20 == 19:
      curr_lr = 0.5 * curr_lr
      for g in optimizer.param_groups:
        g['lr'] = curr_lr


  # (OPTIONAL) Save trained model:
  #PATH = './cifar_net.pt'
  #torch.save(net.state_dict(), PATH)

  # (OPTIONAL) Load saved model
  #net.load_state_dict(torch.load(PATH))
  #net.to(device)

  results_df = pd.DataFrame({"epoch": epoch_list, "training_loss": loss_epoch_list, "test_acc": test_acc_list, "epoch_train_time": epoch_train_time_list, "total_train_time": total_train_time_list, "lr": lr_list, })
  return results_df

seed_experiment(42)
use_half_all=False
results_df = run_experiment(MAX_EPOCHS=3)
results_df

Seeded everything with seed: 42
using Tesla T4
Seeded everything with seed: 42
Files already downloaded and verified
Files already downloaded and verified


INFO:root:Accuracy of plane :  0 %
INFO:root:Accuracy of   car :  0 %
INFO:root:Accuracy of  bird :  0 %
INFO:root:Accuracy of   cat :  0 %
INFO:root:Accuracy of  deer :  0 %
INFO:root:Accuracy of   dog :  0 %
INFO:root:Accuracy of  frog :  0 %
INFO:root:Accuracy of horse :  0 %
INFO:root:Accuracy of  ship :  0 %
INFO:root:Accuracy of truck :  0 %
INFO:root:
Test set: Average loss: 0.0007, Accuracy: 0/10000 (0%)

INFO:root:Train Epoch: 1 [0 (0%)]	Loss: 6.921206
INFO:root:Train Epoch: 1 [32000 (64%)]	Loss: 1.603750
INFO:root:Accuracy of plane : 46 %
INFO:root:Accuracy of   car : 71 %
INFO:root:Accuracy of  bird : 41 %
INFO:root:Accuracy of   cat :  1 %
INFO:root:Accuracy of  deer :  0 %
INFO:root:Accuracy of   dog : 66 %
INFO:root:Accuracy of  frog :  9 %
INFO:root:Accuracy of horse : 30 %
INFO:root:Accuracy of  ship : 61 %
INFO:root:Accuracy of truck : 43 %
INFO:root:
Test set: Average loss: 0.0002, Accuracy: 3714/10000 (37%)

INFO:root:Train Epoch: 2 [0 (0%)]	Loss: 1.908351
INFO:root:

,epoch,training_loss,test_acc,epoch_train_time,total_train_time,lr
0,0,-1.000000,0.00,-1.000000,-1.000000,-1.00
1,1,1.873167,37.14,47.679572,47.679572,0.01
2,2,1.332349,57.12,47.487087,95.166659,0.01


In [8]:
seed_experiment(42)
use_half_all=True
results_df = run_experiment(MAX_EPOCHS=3)
use_half_all=False
results_df

Seeded everything with seed: 42
Files already downloaded and verified
Files already downloaded and verified


INFO:root:Accuracy of plane :  0 %
INFO:root:Accuracy of   car :  0 %
INFO:root:Accuracy of  bird :  0 %
INFO:root:Accuracy of   cat :  0 %
INFO:root:Accuracy of  deer :  0 %
INFO:root:Accuracy of   dog :  0 %
INFO:root:Accuracy of  frog :  0 %
INFO:root:Accuracy of horse :  0 %
INFO:root:Accuracy of  ship :  0 %
INFO:root:Accuracy of truck :  0 %
INFO:root:
Test set: Average loss: 0.0007, Accuracy: 0/10000 (0%)

INFO:root:Train Epoch: 1 [0 (0%)]	Loss: 6.921875
INFO:root:Train Epoch: 1 [32000 (64%)]	Loss: 1.545898
INFO:root:Accuracy of plane : 23 %
INFO:root:Accuracy of   car : 87 %
INFO:root:Accuracy of  bird : 21 %
INFO:root:Accuracy of   cat :  5 %
INFO:root:Accuracy of  deer :  0 %
INFO:root:Accuracy of   dog : 64 %
INFO:root:Accuracy of  frog : 67 %
INFO:root:Accuracy of horse : 44 %
INFO:root:Accuracy of  ship : 59 %
INFO:root:Accuracy of truck : 43 %
INFO:root:
Test set: Average loss: 0.0002, Accuracy: 4161/10000 (42%)

INFO:root:Train Epoch: 2 [0 (0%)]	Loss: 1.933594
INFO:root:

,epoch,training_loss,test_acc,epoch_train_time,total_train_time,lr
0,0,-1.000000,0.00,-1.000000,-1.000000,-1.00
1,1,1.987305,41.61,45.613671,45.613671,0.01
2,2,1.456055,56.53,45.514052,91.127722,0.01


In [9]:
seed_experiment(42)
use_amp=True
results_df = run_experiment(MAX_EPOCHS=3)
use_amp=False
results_df

Seeded everything with seed: 42
Files already downloaded and verified
Files already downloaded and verified


INFO:root:Accuracy of plane :  0 %
INFO:root:Accuracy of   car :  0 %
INFO:root:Accuracy of  bird :  0 %
INFO:root:Accuracy of   cat :  0 %
INFO:root:Accuracy of  deer :  0 %
INFO:root:Accuracy of   dog :  0 %
INFO:root:Accuracy of  frog :  0 %
INFO:root:Accuracy of horse :  0 %
INFO:root:Accuracy of  ship :  0 %
INFO:root:Accuracy of truck :  0 %
INFO:root:
Test set: Average loss: 0.0007, Accuracy: 0/10000 (0%)

INFO:root:Train Epoch: 1 [0 (0%)]	Loss: 6.921195
INFO:root:Train Epoch: 1 [32000 (64%)]	Loss: 1.663948
INFO:root:Accuracy of plane : 47 %
INFO:root:Accuracy of   car : 85 %
INFO:root:Accuracy of  bird : 33 %
INFO:root:Accuracy of   cat :  1 %
INFO:root:Accuracy of  deer :  0 %
INFO:root:Accuracy of   dog : 69 %
INFO:root:Accuracy of  frog : 16 %
INFO:root:Accuracy of horse : 44 %
INFO:root:Accuracy of  ship : 48 %
INFO:root:Accuracy of truck : 22 %
INFO:root:
Test set: Average loss: 0.0002, Accuracy: 3694/10000 (37%)

INFO:root:Train Epoch: 2 [0 (0%)]	Loss: 1.901588
INFO:root:

,epoch,training_loss,test_acc,epoch_train_time,total_train_time,lr
0,0,-1.000000,0.00,-1.000000,-1.000000,-1.00
1,1,1.884535,36.94,67.278944,67.278944,0.01
2,2,1.513227,54.05,67.359797,134.638741,0.01
